In [1]:
import pickle
import torch 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
# from torchtext.data import Field, BucketIterator
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm
from scripts import *
from Seq2seq import *

file_path_alignments = './gec-datasets/alignments/'

cpu


In [ ]:
target_to_index = None
with open("./pickle_files/target2index.pkl", 'rb') as file:
    target_to_index = pickle.load(file)

In [ ]:
source_to_index = None
with open("./pickle_files/source2index.pkl", 'rb') as file:
    source_to_index = pickle.load(file)

In [ ]:
index_to_word_target = None
with open("./pickle_files/index2word_target.pkl", 'rb') as file:
    index_to_word_target = pickle.load(file)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, source, target,source_pad_index = source_to_index['<PAD>'],target_pad_index = target_to_index['<PAD>']):
        self.source = source
        self.target = target
        self.max_length = max(max(len(s) for s in source), max(len(t) for t in target))

        self.source = torch.tensor([s + [source_pad_index] * (self.max_length - len(s)) for s in source])
        self.target = torch.tensor([t + [target_pad_index] * (self.max_length - len(t)) for t in target])

    def __len__(self):
        return len(self.source)

    def __getitem__(self, idx):
        return self.source[idx], self.target[idx]

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [ ]:

model = torch.load('./models/model.pickle', map_location=torch.device('cpu'))

In [ ]:
print(model)

Seq2Seq(
  (encoder): EncoderLSTM(
    (embedding): Embedding(141069, 256)
    (lstm): LSTM(256, 1024, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): DecoderLSTM(
    (embedding): Embedding(111469, 256)
    (lstm): LSTM(256, 1024, num_layers=2, dropout=0.5)
    (out): Linear(in_features=1024, out_features=111469, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)


In [ ]:

def predict(sentence,model):
    model.eval()
    lis = [source_to_index.get(word,source_to_index['<UNK>']) for word in sentence]
    tensor = torch.tensor(lis, dtype=torch.long)
    tensor = tensor.unsqueeze(1)
    tensor = tensor.to(device)
    output = model.inference(tensor,tensor.shape[0])
    output = output.reshape(-1, output.shape[2])
    output = output.argmax(dim=1)
    final = []
    for indx in output:
        final.append(index_to_word_target[indx.item()])

    return final

In [ ]:
def evaluate(model,test_dataset,batch_size):
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

    # GPU Configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    total_acc_eval = 0
    model.eval()
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for source, targets in tqdm(test_dataloader):
            source = source.transpose(0,1)
            targets = targets.transpose(0,1)
            # Move the test input to the device
            source = source.to(device)
            # Move the test label to the device
            targets = targets.to(device)

            # Perform the forward pass
            output = model.inference(source,source.shape[0])

            output = output[1:].reshape(-1, output.shape[2]) 
          
            targets = targets[1:].reshape(-1) 

            predictions = output.argmax(dim=1)
            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy()) 

            # print("shape of argmax ", output.argmax(dim=1).shape)
            acc = (output.argmax(dim=1) == targets).sum().item()
            total_acc_eval += acc
     
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted', zero_division = 0)
    acc = total_acc_eval / (len(test_dataset) * len(test_dataset[0][0]))
    print(f'Evaluation Accuracy: {acc}  Precision: {precision}  Recall: {recall}  F1 Score: {f1}')

In [ ]:
test =  ['<SOS>', 'المضلم', 'فهوا', 'تعلم', 'أشياء', 'خاطئة', '<EOS>']
output = predict(test,model)
print(output)

['<SOS>', 'المظلم', 'فهو', 'تعلم', 'أشياء', 'خاطئة', '<EOS>']


In [ ]:
test_source, test_targets = read_alignments(file_path_alignments + 'zaebuc/zaebuc_dev.txt')
test_source, test_targets = create_windows(test_source, test_targets, window_size = 5)
test_source = convert_to_indicies(test_source, source_to_index)
test_targets = convert_to_indicies(test_targets, target_to_index)
print(test_source[-5],test_targets[-5])

[0, 4627, 33766, 2316, 159, 18976, 1] [0, 4322, 29196, 2196, 163, 16878, 1]


In [ ]:
test_dataset = Dataset(test_source, test_targets)

In [ ]:
evaluate(model,test_dataset,64)

100%|██████████| 17/17 [01:05<00:00,  3.84s/it]


Evaluation Accuracy: 0.6533980582524271  Precision: 0.7870174940650821  Recall: 0.7622977346278317  F1 Score: 0.766382658802384
